In [4]:
%run C://Users/HSS/dd.py

In [5]:
import os

In [16]:
books = pd.read_csv('./goodbooks-10k/books.csv', encoding = 'ISO-8859-1')
ratings = pd.read_csv('./goodbooks-10k/ratings.csv', encoding = 'ISO-8859-1')
book_tags = pd.read_csv('./goodbooks-10k/book_tags.csv', encoding = 'ISO-8859-1')
tags = pd.read_csv('./goodbooks-10k/tags.csv')


In [15]:
books.tail()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...
9999,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,9162,9700,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...


In [10]:
ratings.tail()

,book_id,user_id,rating
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5
981755,10000,51328,1


In [12]:
book_tags.tail()

,goodreads_book_id,tag_id,count
999907,33288638,21303,7
999908,33288638,17271,7
999909,33288638,1126,7
999910,33288638,11478,7
999911,33288638,27939,7


In [17]:
tags.tail()

,tag_id,tag_name
34247,34247,Ｃhildrens
34248,34248,Ｆａｖｏｒｉｔｅｓ
34249,34249,Ｍａｎｇａ
34250,34250,ＳＥＲＩＥＳ
34251,34251,ｆａｖｏｕｒｉｔｅｓ


### book_tags와 tag의 merge

In [18]:
tags_join_df = pd.merge(book_tags, tags, left_on = 'tag_id', right_on = 'tag_id', how = 'inner')

In [19]:
tags_join_df.tail()

,goodreads_book_id,tag_id,count,tag_name
999907,31538635,14690,6,hogwarts
999908,32848471,16149,21,jan-2017
999909,33288638,27821,9,single-mom
999910,33288638,11478,7,fave-author
999911,33288638,27939,7,slowburn


In [21]:
to_read = pd.read_csv('./goodbooks-10k/to_read.csv')

In [23]:
books.authors[:5]

0                 Suzanne Collins
1    J.K. Rowling, Mary GrandPrÃ©
2                 Stephenie Meyer
3                      Harper Lee
4             F. Scott Fitzgerald
Name: authors, dtype: object

### 작가로 Tfidf 실행

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(books['authors'])
tfidf_matrix

<10000x14742 sparse matrix of type '<class 'numpy.float64'>'
	with 43235 stored elements in Compressed Sparse Row format>

In [32]:
# 유사도 측정
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [36]:
titles = books['title']
indices = pd.Series(books.index, index = books['title'])
indices['The Hobbit']

6

In [40]:
sim_scores = list(enumerate(cosine_sim[indices['The Hobbit']]))
sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
sim_scores[:5]

[(6, 1.0), (18, 1.0), (154, 1.0), (160, 1.0), (188, 1.0)]

In [42]:
book_indices = [i[0] for i in sim_scores[:5]]
titles.iloc[book_indices]

6                                             The Hobbit
18     The Fellowship of the Ring (The Lord of the Ri...
154           The Two Towers (The Lord of the Rings, #2)
160    The Return of the King (The Lord of the Rings,...
188    The Lord of the Rings (The Lord of the Rings, ...
Name: title, dtype: object